In [1]:
from transformers import T5Tokenizer, FlaxMT5ForConditionalGeneration
import jax
import jax.numpy as jnp

/Users/sanchitgandhi/venv/lib/python3.8/site-packages/jax/_src/lib/__init__.py:33: UserWarning: JAX on Mac ARM machines is experimental and minimally tested. Please see https://github.com/google/jax/issues/5501 in the event of problems.
  warnings.warn("JAX on Mac ARM machines is experimental and minimally tested. "


In [7]:
# load the Flax model -> set the dtype of the computations to bfloat16
model = FlaxMT5ForConditionalGeneration.from_pretrained("lewtun/tiny-random-mt5", from_pt=True, dtype=jnp.bfloat16)
# convert the model parameters to bfloat16
model.params = model.to_bf16(model.params)

Some weights of FlaxMT5ForConditionalGeneration were not initialized from the model checkpoint at lewtun/tiny-random-mt5 and are newly initialized: {('lm_head', 'kernel')}
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
tokenizer = T5Tokenizer.from_pretrained("lewtun/tiny-random-mt5")

input_context = "The dog"
input_ids = tokenizer(input_context, return_tensors="np").input_ids

In [ ]:
# vanilla generate -> JIT generate 
jit_generate = jax.jit(model.generate, static_argnames=["max_length", "top_k", "do_sample"])

In [8]:
outputs = jit_generate(input_ids=input_ids, max_length=20, top_k=30, do_sample=True).sequences